In [ ]:
#%pip install yfinance numpy statsmodels pandas matplotlib arch pycop

In [ ]:
%reset -f

In [77]:
import yfinance as yf
import pandas as pd
import numpy as np

# List of ticker symbols for each index
tickers = ["^GDAXI", "^AEX", "^N225", "^GSPC", "^GSPTSE"]  # DAX, AEX, Nikkei 225, S&P 500, TSX
index_datasets = {}


# Loop over each ticker symbol to fetch the data
for ticker in tickers:
    # Retrieve the historical data starting from November 20, 1992
    index = yf.Ticker(ticker)
    
    #Save to dictionary
    index_datasets[ticker] = index.history(start="1992-11-20") 


In [163]:
# Initialize a DataFrame to store merged data
merged_data = pd.DataFrame()
k=0
# Loop over each ticker symbol to fetch the data
for ticker in tickers:
    
    index_data = index_datasets[ticker]

    # Extract only the Date and Close columns, reset the index
    close_data = (index_data[['Close']]).reset_index()
    


    # Convert the Date column to Datetime
    close_data['Date'] = pd.to_datetime((close_data['Date'].dt.normalize()).dt.tz_localize(None))

    close_data.rename(columns={'Close': ticker},inplace=True)


    
    # Merge with the previously merged data
    if merged_data.empty:
        merged_data = close_data
    else:
        merged_data = pd.merge_asof(merged_data, close_data, on='Date', direction='nearest')



for ticker in tickers:
    #Forward fill missing data
    merged_data[ticker] = merged_data[ticker].ffill()

#Set Date as index
merged_data.set_index('Date', inplace=True)
#Resample at end of each quarter
merged_data = merged_data.resample("QE").last()

#Compute Log Returns
merged_data[:] = np.log(1+merged_data.pct_change())
#merged_data[:] = merged_data.pct_change()

merged_data.dropna(inplace=True)






In [181]:
from arch import arch_model
from scipy.stats import t

#Fit Student's t-GARCH
def fit_garch(ts):
    model = arch_model(ts, vol="Garch", p=1, q=1, dist="t")
    fitted = model.fit(disp="off")
    residuals = ts - fitted.conditional_volatility  # Approximation for simplicity
    nu = fitted.params["nu"]
    return residuals, nu


nu_dict = {}
residuals_dict = {}

#We scale our returns by a factor of 100 for fitting as per the libraries recommendations
for i in tickers:
    residuals_dict[i], nu_dict[i] = fit_garch(merged_data[i]*100)
    print(i+" Nu: "+str(nu_dict[i]))





^GDAXI Nu: 3.1515891575897332
^AEX Nu: 3.2194036816496427
^N225 Nu: 286.79481857586893
^GSPC Nu: 3.4672641001969513
^GSPTSE Nu: 3.149877563388971


Note the large tails of returns demonstrated by each distibution, except the N225

In [230]:
from scipy.stats import spearmanr
spearman_corr = spearmanr(list(residuals_dict.values()),axis=1)

# Calculate the standard deviations of the residuals
std_devs = []
for i in tickers:
    std_devs.append(np.std(residuals_dict[i]) )

# Calculate the variance-covariance matrix
cov_matrix = np.float64(spearman_corr) * np.outer(std_devs, std_devs)


In [204]:

#Seed
np.random.seed(42) 

#Number Monte Carlo Simulations
n = 1000

simulated_data = np.random.multivariate_normal(np.zeros(len(tickers)),cov_matrix,n)


    

ValueError: cov must be 2 dimensional and square

In [ ]:







# Step 1: Transform the data to uniform margins using ECDF (Empirical CDF)
uniform_data = pd.DataFrame()


# Use empirical CDF for each column (ticker) in the merged data
for ticker in tickers:
    uniform_data[ticker] = merged_data_filled[ticker].rank() / len(merged_data_filled[ticker])

# Step 2: Fit a copula (using Gaussian copula as an example)
copula = StudentTCopula()

# Step 3: Fit the copula to the uniform-transformed data
copula.fit(uniform_data)

# Step 4: Simulate synthetic data from the fitted copula (optional)
simulated_data = copula.sample(len(merged_data_filled))

# Convert the uniform data back to the original scale (inverse of the transformation)
simulated_data_original_scale = pd.DataFrame()

for idx, ticker in enumerate(tickers):
    # Inverse transform (since we used ECDF, we can approximate the inverse by using the quantiles)
    simulated_data_original_scale[ticker] = np.percentile(merged_data_filled[ticker], simulated_data[:, idx] * 100)

# View the simulated data
print(simulated_data_original_scale.head())


In [ ]:
%pip install pycop

Note: you may need to restart the kernel to use updated packages.
